In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 16 10:36:39 2013
 
@author: Florent Delaigue - Script Bruit 3D version 3
"""
 
#==============================================================================
# Paramètres d'entrée du script
#==============================================================================
 
image = 'insb'    # Nom de la séquence à tester
dossier_image = r'D:\Documents and Settings\t0143844\Bureau\Bruit 3D\Banque_Images\\'
dossier_resultat = r'D:\Documents and Settings\t0143844\Bureau\Analyse_Bruit3D\\'
pix_def = 0          # Critère (en %) de mise à niveau des pixels défectueux : 0 par défaut
nb_sorties = 0       # Nombre de sorties vidéos : 2, 4 ou 8, sinon 0 ou 1 par défaut
PSD = 0              # Densité spectrale de bruit : 0 si pas de calcul souhaité (par défaut), 1 sinon
reporting = 0        # Reporting des résultats sur fichier excel : 0 si pas souhaité (par défaut), 1 sinon
affichage_figure = 1 # Affichage des figures lors du lancement du script :  0 si pas souhaité (par défaut), 1 sinon
mode_calcul = 0      # Mode0 : calcul rapide pour petites séquences / Mode1 : calcul long pour grosses séquences si problème de mémoire avec mode0
 
 
#==============================================================================
# Declaration et importation de bibliothèques nécessaires
#==============================================================================
 
import numpy as np
from mahotas.io import freeimage
import Image
import timeit
import os
import sys
import matplotlib.pyplot as plt
from xlwt import Workbook, easyxf
import datetime
#import pdb  pdb.set_trace()
 
plt.close('all') # Fermer toutes les figures ouvertes
now = datetime.datetime.now() # Mesure du temps à ce moment là
tic = timeit.default_timer() # Démarrer le timer
sys.stdout.write('\r')
sys.stdout.write("Traitement realise : 1%")
sys.stdout.flush()
 
 
#==============================================================================
# Tests de validité des paramètres d'entrée
#==============================================================================
 
while pix_def<0 or pix_def>100 :
    print("\nERREUR : le seuil de detection de pixels defectueux doit etre compris entre 0 et 100.")
    print("Veuillez resaisir le seuil de detection desire : ")
    pix_def = input()

while nb_sorties!=0 and nb_sorties!=1 and nb_sorties!=2 and nb_sorties!=4 and nb_sorties!=8 :
    print("\nERREUR : le nombre de sorties videos doit etre egal a 0 (par defaut), 1, 2, 4 ou 8.")
    print("Veuillez resaisir le nombre de sorties video : ")
    nb_sorties = input()

while PSD!=0 and PSD!=1 :
    print("\nERREUR : le mode PSD a saisir est 0 ou 1")
    print("Veuillez resaisir un chiffre correct : ")
    PSD = input()

while reporting!=0 and reporting!=1 :
    print("\nERREUR : le mode reporting a saisir est 0 ou 1")
    print("Veuillez resaisir un chiffre correct : ")
    reporting = input()

while affichage_figure!=0 and affichage_figure!=1 :
    print("\nERREUR : le mode affichage de figures a saisir est 0 ou 1")
    print("Veuillez resaisir usheet3D.write(25,i,'SIGMA_TOT (somme des contrib)'); sheet3D.write(25,j,'%.4f' %sigma_tot)n chiffre correct : ")
    affichage_figure = input()
while mode_calcul!=0 and mode_calcul!=1 :
    print("\nERREUR : le mode de calcul a saisir est 0 ou 1 (sequence volumineuse dans ce dernier cas)")
    print("Veuillez resaisir le mode de calcul desire : ")
    mode_calcul= input()
 
 
#==============================================================================
# Définition des chemins d'accès et des fichiers de sortie
#==============================================================================
 
dossier_resultat_final = dossier_resultat + 'Mesure_Bruit3D_%s_%d-%d-%d_%dh%d\\' %(image,now.day,now.month,now.year,now.hour,now.minute)
if os.path.exists(dossier_resultat_final):
    dossier_resultat_final = dossier_resultat + 'Mesure_Bruit3D_%d-%d-%d_%dh%d_(1)\\' %(now.day,now.month,now.year,now.hour,now.minute)  
    if os.path.exists(dossier_resultat_final):
        dossier_resultat_final = dossier_resultat + 'Mesure_Bruit3D_%d-%d-%d_%dh%d_(2)\\' %(now.day,now.month,now.year,now.hour,now.minute)
        if os.path.exists(dossier_resultat_final):
            dossier_resultat_final = dossier_resultat + 'Mesure_Bruit3D_%d-%d-%d_%dh%d_(3)\\' %(now.day,now.month,now.year,now.hour,now.minute)
            if os.path.exists(dossier_resultat_final):
                dossier_resultat_final = dossier_resultat + 'Mesure_Bruit3D_%d-%d-%d_%dh%d_(4)\\' %(now.day,now.month,now.year,now.hour,now.minute)
               if os.path.exists(dossier_resultat_final):
                    dossier_resultat_final = dossier_resultat + 'Mesure_Bruit3D_%d-%d-%d_%dh%d_(5)\\' %(now.day,now.month,now.year,now.hour,now.minute)
os.makedirs(dossier_resultat_final)
   
seq_in = dossier_image + image + ".tif"
seq_out_txt = dossier_resultat_final + "Resultats_Bruit_3D_" + image + ".txt"
if reporting == 1 :
    fichier_reporting_xls = dossier_resultat_final + 'Resultats_bruit3D_%d-%d-%d_%dh%d.xls' %(now.day,now.month,now.year,now.hour,now.minute)
fichier_texte = file(seq_out_txt, "w+")
   
del dossier_image,dossier_resultat,seq_out_txt
 
sys.stdout.write('\r')
sys.stdout.write("Traitement realise : 2%")
sys.stdout.flush()